## Import

In [153]:
import ccxt
import numpy as np
import pandas as pd
from pytz import timezone
from datetime import datetime

## Miscellaneous

In [7]:
RED = '\u001b[1m\u001b[38;5;196m'
GRAY = '\u001b[1m\u001b[38;5;240m'
GREEN = '\u001b[1m\u001b[38;5;46m'
YELLOW = '\u001b[1m\u001b[38;5;11m'
CYAN = '\u001b[1m\u001b[38;5;51m'
RESET = '\u001b[0m'

time = lambda: '[' + GREEN + datetime.now(timezone('Europe/Amsterdam')).strftime('%Y-%m-%d %H:%M:%S') + RESET + ']'

## Register

In [204]:
exchange = ccxt.poloniex()
exchange.apiKey = 'Y88KY4B1-ASMLC2C2-N0HO4J89-9I3PGNB4'
exchange.secret = 'e97e79187579648c9be3801b4e126601fac097e22e347f13963c130519c4b7a1ffb93872a1d493ac94c2dd0c7f84cc103bf06add17547fd60cce75842c8649eb'
exchange.load_markets ()
exchange.enableRateLimit = True

In [172]:
exchange.rateLimit = 1000
exchange.rateLimit

1000

## All Pairs

In [173]:
pairs = list(exchange.symbols)
coins = set(pair.split('/')[0] for pair in pairs)

In [178]:
symbol = 'BTC/USDT'

## Tickers

In [176]:
temp = exchange.fetch_tickers()
v = [(t, temp[t]['baseVolume']) for t in temp]
v[:10]

[('BCN/BTC', 908485823.176483),
 ('BELA/BTC', 1736838.25469511),
 ('BLK/BTC', 1765622.59741136),
 ('BTCD/BTC', 1688.46138303),
 ('Bitmark/BTC', 73159.7289167),
 ('BTS/BTC', 35208366.49636475),
 ('BURST/BTC', 46791792.48199143),
 ('CLAM/BTC', 89556.82501131),
 ('DASH/BTC', 27958.49078986),
 ('DGB/BTC', 768530729.7781521)]

In [179]:
exchange.fetch_ticker(symbol)
# exchange.fetch_tickers()['BTC/USDT']  # not suggested

{'ask': 16543.39696232,
 'average': None,
 'baseVolume': 13845.09765099,
 'bid': 16500.00000024,
 'change': -0.11841831,
 'close': None,
 'datetime': '2017-12-20T02:52:58.875Z',
 'first': None,
 'high': 18971.91794828,
 'info': {'baseVolume': '239896029.04252300',
  'high24hr': '18971.91794828',
  'highestBid': '16500.00000024',
  'id': 121,
  'isFrozen': '0',
  'last': '16500.00000021',
  'low24hr': '15550.00000008',
  'lowestAsk': '16543.39696232',
  'percentChange': '-0.11841831',
  'quoteVolume': '13845.09765099'},
 'last': 16500.00000021,
 'low': 15550.00000008,
 'open': None,
 'percentage': None,
 'quoteVolume': 239896029.042523,
 'symbol': 'BTC/USDT',
 'timestamp': 1513738377875,
 'vwap': None}

## Order Books

In [180]:
order_book = exchange.fetch_order_book(symbol)
for ob in order_book:
    print('{} {}'.format(time(), ob))
    if ob in ['bids', 'asks']:
        for i in order_book[ob]:
            print('{} {}'.format(time(), i))
        continue
    print('{} {}'.format(time(), order_book[ob]))

[2017-12-20 03:53:04] bids
[2017-12-20 03:53:04] [16500.00000011, 0.70409101]
[2017-12-20 03:53:04] [16500.0, 0.08346062]
[2017-12-20 03:53:04] [16480.0, 0.53918956]
[2017-12-20 03:53:04] [16479.9999998, 0.01274272]
[2017-12-20 03:53:04] [16478.99999997, 0.13415845]
[2017-12-20 03:53:04] [16464.7718603, 0.00020545]
[2017-12-20 03:53:04] [16462.53746248, 0.0008483]
[2017-12-20 03:53:04] [16450.00000006, 0.00834044]
[2017-12-20 03:53:04] [16450.0, 0.31313069]
[2017-12-20 03:53:04] [16446.01579925, 0.03657203]
[2017-12-20 03:53:04] [16445.00000006, 0.47189653]
[2017-12-20 03:53:04] [16445.0, 0.23157603]
[2017-12-20 03:53:04] [16443.8266621, 0.00020572]
[2017-12-20 03:53:04] [16440.0, 0.25]
[2017-12-20 03:53:04] [16439.9999999, 0.2]
[2017-12-20 03:53:04] [16434.0, 0.00056469]
[2017-12-20 03:53:04] [16430.0, 0.06086427]
[2017-12-20 03:53:04] [16421.4737749, 0.000206]
[2017-12-20 03:53:04] [16410.9128647, 0.00020613]
[2017-12-20 03:53:04] [16408.99999994, 0.00889756]
[2017-12-20 03:53:04] [1

## OCHLV Data

Format as below:
```
[
    [
        1504541580000, // UTC timestamp in milliseconds
        4235.4,        // (O)pen price
        4240.6,        // (H)ighest price
        4230.0,        // (L)owest price
        4230.7,        // (C)losing price
        37.72941911    // (V)olume
    ],
    ...
]
```

In [181]:
df = pd.DataFrame(exchange.fetch_ohlcv(symbol, '1d'), columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df.time = df.time.apply(lambda x: datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
df = df.set_index('time')
df.head()

,open,high,low,close,volume
time,,,,,
2015-02-19 01:00:00,0.330000,0.330,225.00,244.00,46.276313
2015-02-20 01:00:00,240.250118,245.000,240.25,240.25,55.894897
2015-02-21 01:00:00,245.000000,245.000,245.00,245.00,14.722239
2015-02-22 01:00:00,245.000000,249.000,235.00,235.00,129.121248
2015-02-23 01:00:00,235.000002,235.001,235.00,235.00,96.567562


## Balances

In [182]:
balances = exchange.fetch_balance()['total']
balances = {c: balances[c] for c in balances if balances[c]}
balances

{'BCH': 2.968e-05,
 'BLK': 0.07283513,
 'BTC': 5.69e-06,
 'CVC': 1.38025015,
 'DASH': 0.00011444,
 'ETC': 0.00319706,
 'ETH': 0.00012825,
 'GAS': 0.02286864,
 'GNO': 0.00030257,
 'GNT': 0.27630823,
 'LSK': 0.00706307,
 'LTC': 0.00039016,
 'MAID': 0.15630676,
 'NXT': 1.78742,
 'OMG': 0.02032775,
 'REP': 0.00111677,
 'STEEM': 0.2255384,
 'STR': 0.39852233,
 'USDT': 38.47432874,
 'XMR': 0.00040302,
 'XRP': 0.15774399,
 'ZEC': 0.00022112,
 'ZRX': 0.14084736}

## Orders

In [185]:
symbol2 = 'CVC/BTC'
symbol2 in exchange.symbols

True

**Market Order**

A sad news is...

In [186]:
try:
    exchange.create_market_buy_order(symbol2, balances[symbol2.split('/')[0]])
except Exception as e:
    print(RED + str(e) + '!')

poloniex allows limit orders only!


**Limit Order**

In [202]:
from poloniex import Poloniex

In [206]:
agent = Poloniex(apiKey, secret)

In [280]:
agent.returnTicker()['BTC_CVC']['lowestAsk'], agent.returnTicker()['BTC_CVC']['lowestAsk'] * 1.0025 * 1.0002, agent.returnTicker()['BTC_CVC']['highestBid'], agent.returnTicker()['BTC_CVC']['highestBid'] * (1-.0025) * (1-.0002)

(3.214e-05, 3.2226794069999993e-05, 3.196e-05, 3.187372398e-05)

In [282]:
exchange.fetch_order_book(symbol2)['asks'][0], exchange.fetch_order_book(symbol2)['bids'][0]

([3.214e-05, 684.4], [3.196e-05, 92.07725857])

In [225]:
2000 / 17000 / 3.168e-5

3713.606654783125

In [227]:
balances[symbol2.split('/')[0]]

1.38025015

In [201]:
exchange.create_order(symbol2, amount=balances[symbol2.split('/')[0]], price=exchange.fetch_order_book(symbol2)['bids'][0][0] * (1 - 0.0025))
# exchange.create_limit_sell_order('', amount)

TypeError: create_order() missing 2 required positional arguments: 'type' and 'side'

**Cancel An Order**

In [ ]:
exchange.cancel_order ('order_id')

## Withdraw

In [ ]:
exchange.withdraw(currency, amount, address)